In [5]:
#Andy - my comments below explaining what's happening here.


from dylan.payoff import VanillaPayoff, call_payoff, put_payoff
from dylan.engine import BinomialPricingEngine, EuropeanBinomialPricer
from dylan.marketdata import MarketData
from dylan.option import Option
import numpy as np


def AmericanBinomialPricer(pricingengine, option, data):
    expiry = option.expiry
    strike = option.strike
    (spot, rate, volatility, dividend) = data.get_data()
    steps = pricingengine.steps
    nodes = steps + 1
    dt = expiry / steps 
    u = np.exp((rate * dt) + volatility * np.sqrt(dt)) 
    d = np.exp((rate * dt) - volatility * np.sqrt(dt))
    pu = (np.exp(rate * dt) - d) / (u - d)
    pd = 1 - pu
    disc = np.exp(-rate * dt)
    dpu = disc * pu
    dpd = disc * pd

    Ct = np.zeros(nodes)
    St = np.zeros(nodes)

    #This is where all the magic happens
    #Here we are building the tree like normal
    for i in range(nodes):
        St[i] = spot * (u ** (steps - i)) * (d ** i)
        Ct[i] = option.payoff(St[i])

    ########################################################
    # Now we need to traverse back through the tree looking for the highest payoff
    # We can't just price end-nodes here, we need to find the highest payoff
    # Wherever we find the highest payoff in each step, we store it, and then move to the next step
    # If the old payoff is higher than the new, we move it up in the array.  If the newer node payoff is higher
    # we store that new value.
    # As we move towards the beginning of the tree, the highest payoff value will keep moving forward in the array.
    # In the end, the highest value is stored in the first value of the array, and will be returned.
    # For example, if the highest payoff in a given tree happens to be in a terminal node, then when this looop is done
    # the entire array will have the same value, because as we walk back in the tree towards the beginning, that
    # value will keep overwriting the lower values, until it is sitting in the first value in the array, and returned.
    # pretty slick.
    for i in range((steps - 1), -1, -1):
        for j in range(i+1):
            Ct[j] = dpu * Ct[j] + dpd * Ct[j+1]
            St[j] = St[j] / u
            Ct[j] = np.maximum(Ct[j], option.payoff(St[j]))

    # As we moved back through the tree, we kept storing the highest payoff, until it was stored in the first value
    # in the array, Ct[0]
    return Ct[0]

def main():
    spot = 41.0
    strike = 40.0
    rate = 0.08
    volatility = 0.30
    expiry = 1.0
    steps = 3
    dividend = 0.0

    the_call = VanillaPayoff(expiry, strike, call_payoff)
    the_bopm = BinomialPricingEngine(steps, EuropeanBinomialPricer)
    the_data = MarketData(rate, spot, volatility, dividend)

    am_pricer = BinomialPricingEngine(steps, AmericanBinomialPricer)
    
    the_option = Option(the_call, the_bopm, the_data)
    am_option = Option(the_call, am_pricer, the_data)
    fmt = "The european call option price is {0:0.3f}"
    print(fmt.format(the_option.price()))
    fmt2 = "The american call option price is {0:0.3f}"
    print(fmt2.format(am_option.price()))


    
if __name__ == "__main__":
    main()


The european call option price is 7.074
The american call option price is 7.074
